# Setup

## import libraries and packages

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os
import cv2
import numpy as np
from random import shuffle
from keras.utils import np_utils
from pathlib import Path
from typing import Union
from multiprocessing import Pool
import keras
import pandas as pd
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## setup constants

In [2]:
TRAIN = Path('data/ASL_Alphabet/asl_alphabet_train/asl_alphabet_train')
TEST = Path('data/ASL_Alphabet/asl_alphabet_test/asl_alphabet_test')
RESIZED_TRAIN = Path('data/ASL_Alphabet/asl_alphabet_train/asl_alphabet_train_resized')
RESIZED_TEST = Path('data/ASL_Alphabet/asl_alphabet_test/asl_alphabet_test_resized')
RESIZED_NPY = Path('data/ASL_Alphabet/npy')
NEW_SIZE = (224,224)
MODEL = Path('/home/yoav/PycharmProjects/ITC/hackathon/Team10/model.json')
MODEL_WEIGHTS = Path('/home/yoav/PycharmProjects/ITC/hackathon/Team10/weights.h5')
RANDOM_STATE = 42
VALIDATION_SIZE = 0.2

## generate output paths

In [3]:
os.makedirs(RESIZED_TRAIN, exist_ok=True)
os.makedirs(RESIZED_TEST, exist_ok=True)
os.makedirs(RESIZED_NPY, exist_ok=True)

## helper functions

In [4]:
def sub_dirs(directory: Path):
    sub_lst = []
    for sub_dir in directory.glob('*'):
        if sub_dir.is_dir():
            sub_lst.append(sub_dir)
    return sub_lst

def resize_img(img: Union[str, Path], new_img: Union[str, Path], 
               new_size=(224,224)):
    image = cv2.imread(str(img))
    resized_image = cv2.resize(image,new_size)
    cv2.imwrite(str(new_img), resized_image)
    
def pool_4_train_set(input_dirs: list, output_dir: Path, new_size: tuple) -> list:
    features, labels, pool_jobs = [], [], []
    for input_sub in sub_dirs(input_dirs):
        label = input_sub.name
        output_sub = output_dir / label
        os.makedirs(output_sub, exist_ok=True)
        for img in input_sub.glob('*.jpg'):
            new_img = output_sub / img.name
            pool_jobs.append((img, new_img, new_size))
            features.append(str(new_img)), labels.append(label)
    return features, labels, pool_jobs

def pool_4_test_set(input_dir: list, output_dir: Path, new_size: tuple) -> list:
    features, labels, pool_jobs = [], [], []
    for img in input_dir.glob('*.jpg'):
        label = img.name.split('_')[0]
        output_sub = output_dir / label
        os.makedirs(output_sub, exist_ok=True)
        new_img = output_sub / img.name
        pool_jobs.append((img, new_img, new_size))
        features.append(str(new_img)), labels.append(label)
    return features, labels, pool_jobs

def scale_pool(img_lst, by: int = 255):
    return [(img, by) for img in img_lst]
    
def scale(img_dir: np.array, by: int = 255):
    return cv2.imread(str(img_dir))/255

# preprocessing

## creating resize image jobs

In [5]:
X_train, y_train, pool_train = pool_4_train_set(TRAIN, RESIZED_TRAIN, NEW_SIZE)
X_test, y_test, pool_test = pool_4_test_set(TEST, RESIZED_TEST, NEW_SIZE)

## split train to validation set

In [6]:
X_train, X_val, y_train, y_val, pool_train, pool_val = train_test_split(X_train, y_train, pool_train, test_size=VALIDATION_SIZE, random_state=RANDOM_STATE, stratify=y_train)

## execute image resize and scale

In [7]:
with Pool() as p: p.starmap(resize_img, pool_train)
with Pool() as p: p.starmap(resize_img, pool_val)
with Pool() as p: p.starmap(resize_img, pool_test)

## convert 2 one hot encoding

In [8]:
label_lst = pd.get_dummies(y_train).columns.tolist()
y_train_cat = pd.get_dummies(y_train).values
y_val_cat = pd.get_dummies(y_val).values
y_test_cat = pd.get_dummies(y_test).values

## read images to np.array and scale

In [9]:
X_train = np.array([cv2.imread(str(img)) for img in X_train])
X_val = np.array([cv2.imread(str(img)) for img in X_val])
X_test = np.array([cv2.imread(str(img)) for img in X_test])

## save numpy object to disk

In [10]:
np.save(str(RESIZED_NPY/'X_train.npy'), X_train)
np.save(str(RESIZED_NPY/'X_val.npy'), X_val)
np.save(str(RESIZED_NPY/'X_test.npy'), X_test)
np.save(str(RESIZED_NPY/'y_train.npy'), y_train_cat)
np.save(str(RESIZED_NPY/'y_val.npy'), y_val_cat)
np.save(str(RESIZED_NPY/'y_test.npy'), y_test_cat)

## load numpy object to disk

In [11]:
X_train=np.load(str(RESIZED_NPY/'X_train.npy'))
Y_train=np.load(str(RESIZED_NPY/'y_train.npy'))
X_val=np.load(str(RESIZED_NPY/'X_val.npy'))
Y_val=np.load(str(RESIZED_NPY/'y_val.npy'))
X_test=np.load(str(RESIZED_NPY/'X_test.npy'))
Y_test=np.load(str(RESIZED_NPY/'y_test.npy'))

# Modeling

## creating a sequential model

In [12]:
model = Sequential()
# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling 
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())
# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())
# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())
# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())
# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())
# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())
# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.6))
# Batch Normalisation
model.add(BatchNormalization())
# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())
# Output Layer
model.add(Dense(29))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_1 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

## Compile 

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## some callbacks

In [14]:
checkpoint = ModelCheckpoint("Checkpoint/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, 
save_best_only=False, save_weights_only=False, mode='auto')
callback = EarlyStopping(monitor='loss', patience=5)

# Train

In [15]:
model.fit(X_train, Y_train, batch_size=32, epochs=50, verbose=1,validation_data=(X_val, Y_val), shuffle=True,callbacks=[callback, checkpoint])

Epoch 1/50
2175/2175 [==============================] - 1298s 596ms/step - loss: 2.6050 - accuracy: 0.2857 - val_loss: 1.3996 - val_accuracy: 0.6521
Epoch 2/50
2175/2175 [==============================] - 1296s 596ms/step - loss: 0.4656 - accuracy: 0.8473 - val_loss: 0.4323 - val_accuracy: 0.8683
Epoch 3/50
2175/2175 [==============================] - 1224s 563ms/step - loss: 0.1980 - accuracy: 0.9364 - val_loss: 0.4775 - val_accuracy: 0.8687
Epoch 4/50
2175/2175 [==============================] - 1248s 574ms/step - loss: 0.1422 - accuracy: 0.9575 - val_loss: 1.5815 - val_accuracy: 0.7481
Epoch 5/50
2175/2175 [==============================] - 1332s 613ms/step - loss: 0.1022 - accuracy: 0.9701 - val_loss: 0.0667 - val_accuracy: 0.9811
Epoch 6/50
2175/2175 [==============================] - 1255s 577ms/step - loss: 0.0729 - accuracy: 0.9788 - val_loss: 0.0233 - val_accuracy: 0.9928
Epoch 7/50
2175/2175 [==============================] - 1215s 559ms/step - loss: 0.0653 - accuracy: 0.9809

# save model and weights to disk

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open(str(MODEL), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(str(MODEL_WEIGHTS))

# Testing

In [37]:
accu = np.mean(np.argmax(y_test_cat, axis=1) == np.argmax(model.predict(X_test), axis=1))
print('accuracy score over test = {}'.format(np.round(accu, 2)))

accuracy score over test = 0.93


In [55]:
list(Path('real_test/').glob('*'))

[PosixPath('real_test/IMG_20210118_143810.jpg'),
 PosixPath('real_test/IMG_20210118_143318.jpg'),
 PosixPath('real_test/IMG_20210118_143928.jpg'),
 PosixPath('real_test/IMG_20210118_143323.jpg'),
 PosixPath('real_test/IMG_20210118_143328.jpg'),
 PosixPath('real_test/IMG_20210118_143925.jpg'),
 PosixPath('real_test/IMG_20210118_143441.jpg'),
 PosixPath('real_test/IMG_20210118_143419.jpg'),
 PosixPath('real_test/IMG_20210118_143420.jpg'),
 PosixPath('real_test/IMG_20210118_143946.jpg')]

In [ ]:
def resize_img(img: Union[str, Path], new_img: Union[str, Path], 
               new_size=(224,224)):
    image = cv2.imread(str(img))
    resized_image = cv2.resize(image,new_size)
    cv2.imwrite(str(new_img), resized_image)

In [57]:
os.makedirs('real_test_resize')

In [32]:
for img in Path('real_test/').glob('*'):
    resize_img(img, Path('real_test_resize') / Path(img).name)

In [33]:
real_lst = list(Path('real_test_resize/').glob('*'))

In [34]:
real_test = [cv2.imread(str(img)) for img in real_lst]

In [35]:
[my_lst[item] for item in list(np.argmax(model.predict(np.array(real_test).reshape(-1,224,224,3)), axis=1))]

['Q', 'F', 'Y', 'F', 'Y', 'C', 'C', 'U', 'N', 'F', 'F', 'S', 'F', 'Y', 'K']

In [103]:
X_test[0].reshape(1,224,224,3).shape

(1, 224, 224, 3)

In [25]:
my_lst = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

In [36]:
real_lst

[PosixPath('real_test_resize/IMG_20210118_143810.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143318.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143928.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143323.jpg'),
 PosixPath('real_test_resize/E.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143328.jpg'),
 PosixPath('real_test_resize/C.jpg'),
 PosixPath('real_test_resize/D.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143925.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143441.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143419.jpg'),
 PosixPath('real_test_resize/A.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143420.jpg'),
 PosixPath('real_test_resize/IMG_20210118_143946.jpg'),
 PosixPath('real_test_resize/B.jpg')]

- G -> C
-  -> B
-  -> B
- G -> F
- C -> C
-  -> I
- G -> F
-  -> C
-  -> B
- K -> F

In [ ]:
https://meet.google.com/mwc-dpjv-ttu

In [ ]:

class_dic = {"L":0,"B":1,"C":2,"V":3,"W":4}

print("Unpacked Dataset")

image_list = []
image_class = []

path = "../asl_alphabet_data"


data_folder_path = "../asl_alphabet_data"


files = os.listdir(data_folder_path)

print(type(files))

required_files = []

symbols = ['B','C','L','V','W']

for i in files:
    if i[0] in symbols:
        required_files.append(i)


for i in range(10):
	shuffle(required_files)

print("Shuffled Data Files")
  


# print(type(files))
print(len(required_files))

class_count = {'L':0,'B':0,'C':0,'V':0,'W':0}

X = []
Y = []
X_val = []
Y_val = []
X_test = []
Y_test = []
unique_list=[]

for file_name in required_files:
#  print(file_name)
  label = file_name[0]
  if label not in unique_list:
    print(label)
    unique_list.append(label)
  path = data_folder_path+'/'+file_name
  image = cv2.imread(path)
  resized_image = cv2.resize(image,(224,224))
  if class_count[label]<2700:
    class_count[label]+=1
    X.append(resized_image)
    Y.append(class_dic[label])
  #elif class_count[label]>=2000 and class_count[label]<2750:
  #  class_count[label]+=1
  #  X_val.append(resized_image)
  #  Y_val.append(class_dic[label])
  else:
    X_test.append(resized_image)
    Y_test.append(class_dic[label])

print(len(unique_list))
	
Y = np_utils.to_categorical(Y)
#Y_val = np_utils.to_categorical(Y_val)
Y_test = np_utils.to_categorical(Y_test)

print(len(Y))
#print(len(Y_val))
print(len(Y_test))

print(len(X))
#print(len(X_val))
print(len(X_test))

npy_data_path  = "Numpy"

np.save(npy_data_path+'/train_set.npy',X)
np.save(npy_data_path+'/train_classes.npy',Y)

#np.save(npy_data_path+'/validation_set.npy',X_val)
#np.save(npy_data_path+'/validation_classes.npy',Y_val)

np.save(npy_data_path+'/test_set.npy',X_test)
np.save(npy_data_path+'/test_classes.npy',Y_test)

print("Data pre-processing Success!")

#get train and validation sets
# npy_data_path  = "/content/drive/My Drive/ASL_Colab/Image_To_Numpy_Data"

X_train=np.load(npy_data_path+"/train_set.npy")
Y_train=np.load(npy_data_path+"/train_classes.npy")

#X_valid=np.load(npy_data_path+"/validation_set.npy")
#Y_valid=np.load(npy_data_path+"/validation_classes.npy")

X_test=np.load(npy_data_path+"/test_set.npy")
Y_test=np.load(npy_data_path+"/test_classes.npy")

X_test.shape

import keras
from keras.optimizers import SGD
from keras.models import Sequential
from keras.applications import VGG16
from keras.preprocessing import image
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D

print("Imported Network Essentials")

#Load the VGG model
image_size=224
vgg_base = VGG16(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

#initiate a model
model = Sequential()

#Add the VGG base model
model.add(vgg_base)

#Add new layers
model.add(Flatten())

model.add(Dense(8192, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))



# (4) Compile 
sgd = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
checkpoint = keras.callbacks.ModelCheckpoint("Weights/weights.{epoch:02d}-{val_loss:.2f}.hdf5", monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
# (5) Train
model.fit(X_train/255.0, Y_train, batch_size=32, epochs=15, verbose=1,validation_data=(X_test/255.0,Y_test/255.0), shuffle=True,callbacks=[checkpoint])

# serialize model to JSON
model_json = model.to_json()
with open("Model/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("Model/model_weights.h5")
print("Saved model to disk")
